# Question 1

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from numpy.linalg import inv
from numpy.linalg import norm
from numpy.typing import NDArray
from scipy import io, integrate, linalg, signal
from scipy.linalg import lu_factor, lu_solve

In [6]:
def func_1(input_vec):
    x, y = input_vec#2d x array
    f1 = x**2 +y**2 -4
    f2 = np.exp(x)+ y -1

    return np.array([f1,f2], dtype=float)

In [12]:
def jac_1(x):
    return np.array([[2*x[0], 2*x[1]],
                     [np.exp(x[0]), 1]])

In [42]:
def approximate_jacobian(f, x, h=1e-8):
    n = len(x)
    fx = f(x)
    m = len(fx)
    J = np.zeros((m, n))
    for j in range(n):
        x1 = x.copy()
        x1[j] += h
        fx1 = f(x1)
        J[:, j] = (fx1 - fx) / h
    return J

In [8]:
input_vec_1=np.array([1, 1])
input_vec_2=np.array([1, -1])
input_vec_3=np.array([0, 0])

In [45]:
def Newton_Q1(x0,tol,Nmax):

    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''

    for its in range(Nmax):
       J = jac_1(x0)
       if np.linalg.det(J) == 0:
            print("Non-invertible Jacobian")
            return None, 1, None
       Jinv = inv(J)
       F = func_1(x0)
       
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier, its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]

def LazyNewton_Q1(x0,tol,Nmax):

    ''' Lazy Newton = use only the inverse of the Jacobian for initial guess'''
    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''

    J = jac_1(x0)
    if np.linalg.det(J) == 0:
        print("Non-invertible Jacobian")
        return None, 1, None
    Jinv = inv(J)

    for its in range(Nmax):

       F = func_1(x0)
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier,its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]  

     

In [47]:

def Broyden_q1(f, B0, x0, tol, nmax, Bmat='Id', verb=False):
    """
    Broyden's method for solving f(x)=0 in R^n.

    Parameters:
        f : function
            Function f: R^n -> R^n.
        B0 : ndarray or None
            Initial approximation matrix. It is ignored if Bmat=='Id'.
        x0 : ndarray
            Initial guess.
        tol : float
            Tolerance for convergence.
        nmax : int
            Maximum number of iterations.
        Bmat : str, optional
            Specifies how to set B0:
            'fwd' : B0 is an approximation of Jf(x0)
            'inv' : B0 is an approximation of the inverse of Jf(x0)
            'Id'  : B0 is taken as the identity matrix
        verb : bool, optional
            If True, prints iteration information.

    Returns:
        x : ndarray
            Approximate solution.
        ier : int
            Error flag (0 if converged, 1 otherwise).
        it : int
            Number of iterations performed.
    """
    n = len(x0)
    x = x0.copy()

    # Initialize B0 based on the chosen option
    if Bmat == 'fwd':
        if B0 is None:
            B0 = approximate_jacobian(f, x0)
    elif Bmat == 'inv':
        if B0 is None:
            J0 = approximate_jacobian(f, x0)
            B0 = inv(J0)
    elif Bmat == 'Id':
        B0 = np.eye(n)
    else:
        raise ValueError("Unknown Bmat option. Use 'fwd', 'inv', or 'Id'.")

    fx = f(x)

    for it in range(1, nmax+1):
        # Depending on whether B0 approximates the Jacobian or its inverse, 
        # compute the update dx.
        if Bmat == 'inv':
            dx = -B0.dot(fx)
        else:
            # If B0 approximates the Jacobian, solve B0 dx = -f(x)
            dx = np.linalg.solve(B0, -fx)
        
        x_new = x + dx
        
        if norm(dx) < tol:
            if verb:
                print(f"Converged in {it} iterations.")
            return x_new, 0, it
        
        f_new = f(x_new)
        y = f_new - fx  # Change in function values
        
        # Broyden update for B0
        if Bmat in ['fwd', 'Id']:
            # Forward (Jacobian) update:
            dx_norm_sq = np.dot(dx, dx)
            if dx_norm_sq == 0:
                if verb:
                    print("Zero denominator in forward update; terminating.")
                return x_new, 1, it
            B0 = B0 + np.outer((y - B0.dot(dx)), dx) / dx_norm_sq
        elif Bmat == 'inv':
            # Inverse update (Sherman-Morrison formula):
            denom = np.dot(dx, B0.dot(f_new))
            if abs(denom) < 1e-12:
                if verb:
                    print("Small denominator in inverse update; terminating.")
                return x_new, 1, it
            B0 = B0 + np.outer((dx - B0.dot(y)), (dx.dot(B0))) / denom
        
        # Prepare for next iteration
        x = x_new
        fx = f_new
        
        # if verb:
        #     print(f"Iteration {it}: x = {x}, f(x) = {fx}")

    return x, 1, nmax


In [34]:
# initial_guesses = np.array([(1,1), (1,-1), (0,0)])
tol = 1e-7
max_iter = 100

In [56]:
Broyden_q1(func_1, None, np.array((0,0)), tol, max_iter, Bmat='Id', verb=True)

Iteration 1: x = [4. 0.], f(x) = [12.         53.59815003]
Iteration 2: x = [  1.         -13.39953751], f(x) = [176.54760544 -11.68125568]
Iteration 3: x = [-0.02268209  0.5745922 ], f(x) = [-3.66932933  0.55216541]
Iteration 4: x = [-0.02388757  0.28798907], f(x) = [-3.91649168  0.26438455]
Iteration 5: x = [-0.40094856  5.41534964], f(x) = [25.48677143  5.08503414]
Iteration 6: x = [-0.14276026  0.9816466 ], f(x) = [-3.01598946  0.84860849]
Iteration 7: x = [-0.26770239  1.46381955], f(x) = [-1.78556776  1.22895501]
Iteration 8: x = [-0.61900161  2.26318011], f(x) = [1.50514721 1.8016619 ]
Iteration 9: x = [-0.60270838  1.93124381], f(x) = [0.09296007 1.47857107]
Iteration 10: x = [-0.72420529  1.94023298], f(x) = [0.28897733 1.4249426 ]
Iteration 11: x = [-1.68501805  1.60104464], f(x) = [1.40262976 0.78648572]
Iteration 12: x = [-1.70498058  1.23971339], f(x) = [0.44384806 0.42148931]
Iteration 13: x = [-2.03231445  0.77165492], f(x) = [ 0.72575336 -0.09731318]
Iteration 14: x = [

(array([-1.81626407,  0.8373678 ]), 0, 20)

In [53]:
Broyden_q1(func_1, None, np.array((1,1)), tol, max_iter, Bmat='Id', verb=True)

Iteration 1: x = [ 3.         -1.71828183], f(x) = [ 7.95249244 17.36725509]
Iteration 2: x = [7.5478856  8.21373508], f(x) = [ 120.43602093 1903.94178258]
Iteration 3: x = [ 0.67584982 -0.69867813], f(x) = [-3.0550759   0.26702463]
Iteration 4: x = [ 1.66850374 -1.18359563], f(x) = [0.18480334 3.1206297 ]
Iteration 5: x = [ 1.66292738 -1.20102258], f(x) = [0.20778272 3.07370685]
Iteration 6: x = [ 1.51904447 -1.33642317], f(x) = [0.09352299 2.23143524]
Iteration 7: x = [ 0.65812581 -2.35204652], f(x) = [ 1.9652524  -1.42087696]
Iteration 8: x = [ 1.34714491 -1.42235015], f(x) = [-0.16212065  1.42407778]
Iteration 9: x = [ 1.20206053 -1.54858186], f(x) = [-0.1569447   0.77838333]
Iteration 10: x = [ 0.89394988 -1.85747964], f(x) = [ 0.249377  -0.4127125]
Iteration 11: x = [ 1.04540667 -1.6931624 ], f(x) = [-0.040326    0.15139267]
Iteration 12: x = [ 1.01699005 -1.72058074], f(x) = [-0.00533315  0.04427941]
Iteration 13: x = [ 1.00920798 -1.72703193], f(x) = [0.00114002 0.01639536]
Ite

(array([ 1.00416874, -1.72963729]), 0, 20)

In [54]:
Broyden_q1(func_1, None, np.array((1,-1)), tol, max_iter, Bmat='Id', verb=True)

Iteration 1: x = [ 3.         -1.71828183], f(x) = [ 7.95249244 17.36725509]
Iteration 2: x = [ -1.51943097 -11.58815735], f(x) = [132.59406121 -12.36932097]
Iteration 3: x = [ 0.93873908 -0.98156009], f(x) = [-2.15530873  0.57519545]
Iteration 4: x = [ 0.85458223 -1.14904203], f(x) = [-1.94939164  0.20135022]
Iteration 5: x = [ 0.6470422  -2.00287161], f(x) = [ 0.4301583 -1.0929882]
Iteration 6: x = [ 0.79847782 -1.72852555], f(x) = [-0.3746326  -0.50636971]
Iteration 7: x = [ 0.85976177 -1.76362378], f(x) = [-0.15044085 -0.40102599]
Iteration 8: x = [ 1.02412965 -1.73701705], f(x) = [0.06606977 0.04765373]
Iteration 9: x = [ 1.00278236 -1.73013365], f(x) = [-0.00106509 -0.00427806]
Iteration 10: x = [ 1.00418084 -1.72968017], f(x) = [ 1.72639571e-04 -9.83647181e-06]
Iteration 11: x = [ 1.00416992 -1.72963904], f(x) = [8.45391615e-06 1.47775421e-06]
Iteration 12: x = [ 1.00416874 -1.72963729], f(x) = [6.52259402e-09 7.12905290e-11]
Converged in 13 iterations.


(array([ 1.00416874, -1.72963729]), 0, 13)

In [36]:
print(LazyNewton_Q1(func_1, (1,1), tol, max_iter))
print(LazyNewton_Q1((1,-1), tol, max_iter))
print(LazyNewton_Q1((0,0), tol, max_iter))

[array([nan, nan]), 1, 99]
[array([ 1.00416873, -1.72963726]), 0, 25]
Non-invertible Jacobian
(None, 1, None)


/var/folders/bk/w3kf9k496lv9ntkhhfbltxs80000gn/T/ipykernel_92136/3690288146.py:4: RuntimeWarning: overflow encountered in exp
  f2 = np.exp(x)+ y -1


In [40]:
print(Newton_Q1((1,1), tol, max_iter))
print(Newton_Q1((1,-1), tol, max_iter))
print(Newton_Q1((0,0), tol, max_iter))

[array([-1.81626407,  0.8373678 ]), 0, 7]
[array([ 1.00416874, -1.72963729]), 0, 4]
Non-invertible Jacobian
(None, 1, None)


# Quesiton 2


In [60]:
def func_2(x):
    f1 = x[0] + math.cos(x[0]*x[1]*x[2]) - 1
    f2 = (1 - x[0])**(1/4) + x[1] + 0.05*x[2]**2 - 0.15*x[2] - 1
    f3 = -x[0]**2 - 0.1*x[1]**2 + 0.01*x[1] + x[2] - 1
    return np.array([f1, f2, f3])

def jac_2(x):
    J11 = 1 - math.sin(x[0]*x[1]*x[2]) * (x[1]*x[2])
    J12 = - math.sin(x[0]*x[1]*x[2]) * (x[0]*x[2])
    J13 = - math.sin(x[0]*x[1]*x[2]) * (x[0]*x[1])
    
    J21 = - (1/4) * (1 - x[0])**(-3/4)
    J22 = 1
    J23 = 0.1*x[2] - 0.15
    
    J31 = -2*x[0]
    J32 = -0.2*x[1] + 0.01
    J33 = 1
    
    return np.array([[J11, J12, J13],
                     [J21, J22, J23],
                     [J31, J32, J33]])

In [61]:
def Newton_q2(x0,Nmax=100, tol =1e-6):

    ''' inputs: x0 = initial guess, tol = tolerance, Nmax = max its'''
    ''' Outputs: xstar= approx root, ier = error message, its = num its'''

    for its in range(Nmax):
       J = jac_2(x0)
       Jinv = inv(J)
       F = func_2(x0)
       
       x1 = x0 - Jinv.dot(F)
       
       if (norm(x1-x0) < tol):
           xstar = x1
           ier =0
           return[xstar, ier, its]
           
       x0 = x1
    
    xstar = x1
    ier = 1
    return[xstar,ier,its]


In [71]:
def steepest_descent(x0, Nmax=100, tol=1e-6):
    x = x0.copy()
    for it in range(Nmax):
        fx = func_2(x)
        phi = 0.5 * norm(fx)**2
        if norm(fx) < tol:
            return x, it+1
        grad = jac_2(x).T.dot(fx)
        num = np.dot(grad, grad)
        denom = np.dot(grad, (jac_2(x).T @ jac_2(x)) @ grad)
        if denom == 0:
            alpha = 1e-3
        else:
            alpha = num / denom
        x = x - alpha * grad
    return x, Nmax


In [95]:
def hybrid_method(x0, tol_phase1=5e-2, tol_phase2=1e-6, maxit_phase1=100, maxit_phase2=100):
    x_phase1, it1 = steepest_descent(x0, Nmax= maxit_phase1, tol=tol_phase1)

    x_phase2, ier, it2 = Newton_q2(x_phase1,Nmax=maxit_phase2, tol=tol_phase2)
    
    return x_phase2, it1, it2


In [90]:

x0 = np.array([0.5, 0.5, 0.5])
print("Initial guess:", x0)

# Test Newton's method
start = time.time()
sol_newton, ier,it_newton = Newton_q2(x0, 1e-7, 100)
print("\nNewton's method:")
print("  Solution =", sol_newton)
print("  Iterations =", it_newton)
print("  Residual norm =", norm(func_2(sol_newton)))


Initial guess: [0.5 0.5 0.5]

Newton's method:
  Solution = [-3.89385281e-17  1.00000000e-01  1.00000000e+00]
  Iterations = 4
  Residual norm = 1.1102230246251565e-16


In [91]:
# Test Steepest Descent method
sol_sd, it_sd = steepest_descent(x0, tol=1e-6)
print("\nSteepest Descent method:")
print("  Solution =", sol_sd)
print("  Iterations =", it_sd)
print("  Residual norm =", norm(func_2(sol_sd)))



Steepest Descent method:
  Solution = [7.54089630e-07 1.00000278e-01 1.00000018e+00]
  Iterations = 11
  Residual norm = 7.796339767305405e-07


In [96]:

sol_hybrid, it_phase1, it_phase2 = hybrid_method(x0, tol_phase1=5e-2, tol_phase2=1e-6)
print("\nHybrid (Steepest Descent then Newton) method:")
print("  Intermediate steepest descent iterations =", it_phase1)
print("  Newton iterations after steepest descent =", it_phase2)
print("  Total iterations =", it_phase1 + it_phase2)
print("  Solution =", sol_hybrid)
print("  Residual norm =", norm(func_2(sol_hybrid)))



Hybrid (Steepest Descent then Newton) method:
  Intermediate steepest descent iterations = 4
  Newton iterations after steepest descent = 2
  Total iterations = 6
  Solution = [2.69598361e-17 1.00000000e-01 1.00000000e+00]
  Residual norm = 1.1102230246251565e-16
